In [ ]:
import numpy as np
import sympy as sp
from scipy.linalg import block_diag

The theory behind:

1) Find the eigenvalues

2) For each eigenvalue find the generalized eigenspace. More precisely find such $k$ that $\dim Ker(A-\lambda)^{k}=\dim Ker(A-\lambda)^{k+1}$

3) Starting from the eigenspace with the highest rank compute the following quantity:

$$
d_{k}=\dim H(k,\lambda)-\dim H(k-1,\lambda)
$$

$$
\dim H(k,\lambda)=\dim Ker(A-\lambda)^{k}=size(A)-\dim Im(A-\lambda)^{k}=size(A)-rk(A-\lambda)^{k}
$$

$$
d_{k}=size(A)-rk(A-\lambda)^{k}-size(A)+rk(A-\lambda)^{k-1}=rk(A-\lambda)^{k-1}-rk(A-\lambda)^{k}
$$

Gives the number of generalized eigenvectors of rank $k$. Since $H(k-1,\lambda)\subset H(k,\lambda)$, we can extend the basis of $H(k-1,\lambda)$ with exactly $d_{k}$ vectors to obtain the basis of $H(k,\lambda)$. Those vectors are not in $H(k-1,\lambda)$, hence we indeed obtain generalized eigenvectors.

4) Proposition: each eigenvector induces the Jordan chain

Suppose $v\in H(k,\lambda)$ is a generalized eigenvector. Then:

$$
\sum_{i=0}^{k-1}a_{i}(A-\lambda)^{i}v=0
$$

Applying $(A-\lambda)^{k-1}$:

$$
a_{0}(A-\lambda)^{k-1}v+a_{1}(A-\lambda)^{k}v+\cdots+a_{k-1}(A-\lambda)^{k-2}v=0
$$

Since $v\in Ker(A-\lambda)^{k}$ then $(A-\lambda)^{m}v=0$ for $m\geq k$. All terms vanish except:

$$
a_{0}(A-\lambda)^{k-1}v=0,\ (A-\lambda)^{k-1}v\neq 0\ \Rightarrow a_{0}=0
$$

Repeating this procedure shows $a_{i}=0$ for all $i=0,...,k-1$. Thus $(A-\lambda)^{k-1}v,...,(A-\lambda)v,v$ are linearly independent.

Each $(A-\lambda)^{k-i}v$ is a generalized eigenvector in $H(i,\lambda)$. If $(A-\lambda)^{k-i}v\in H(i-1,\lambda)$, then $(A-\lambda)^{i-1}(A-\lambda)^{k-i}v=(A-\lambda)^{k-1}v=0$, contradicting $v\in H(k,\lambda)$.

5) $d_{k}$ defines the number of Jordan chains of size at least $k$. If $n$ is the current number of chains, the number of new chains is $d_{k}-n$.

Test utilities:

In [ ]:
def generate_nice_matrix(scale, size):
  return sp.Matrix(np.triu(np.random.randint(1, scale, size=(size, size))))

In [ ]:
def ranks(matrix: sp.Matrix):
    ranks = [matrix.shape[0]]  #dim(E(lambda)) = n-dim(Im(A-lambda)) = n-rk(A-lambda)
    current_matrix = sp.eye(matrix.shape[0])
    while True:
        current_rank = current_matrix.rank()
        next_matrix = current_matrix * matrix
        if current_rank == next_matrix.rank():
            break
        ranks.append(next_matrix.rank())
        current_matrix = next_matrix
    return ranks[::-1]


def chains(ranks):
    chain_count = 0
    chains_ = []

    for i in range(len(ranks) - 1):
        d = ranks[i + 1] - ranks[i]
        if d > chain_count:
            chains_.append((d - chain_count, len(ranks) - 1 - i))
            chain_count = d

    return chains_


def jordan_form(matrix: sp.Matrix):
    size = matrix.shape[0]
    blocks = []
    evas = matrix.eigenvals().keys()
    for eva in evas:
        chains_ = chains(ranks(matrix - eva * sp.eye(size)))
        for chain in chains_:
            chain_size = chain[1]
            block = sp.jordan_cell(eva, chain_size)
            blocks.extend([block] * chain[0])
    return sp.Matrix(block_diag(*blocks))


A = sp.Matrix(
    np.array(
        [
            [3, 1, -1, 1, -1],
            [-1, 5, 1, -1, 1],
            [-1, 1, 3, 1, -1],
            [0, 0, 0, 4, 2],
            [-1, 1, 1, -1, 7],
        ]
    )
)

print(jordan_form(A))

Matrix([[2, 0, 0, 0, 0], [0, 6, 1, 0, 0], [0, 0, 6, 0, 0], [0, 0, 0, 4, 0], [0, 0, 0, 0, 4]])


TODO: get rid of block_diag from scipy

In [ ]:
def sympy_to_numpy(matrix : sp.Matrix):
  return np.array(matrix.tolist()).astype(np.float64)

def similar(A : sp.Matrix, B : sp.Matrix):
  if A.shape != B.shape:
    return False

  return (A*P).solve(P*B)

Running the tests:

TODO: make the comparison more robust (may fail due to different ordering of the blocks). Simply can generate change of bases and compare equations $CJC^{(-1)}= C'J'{C'}^{-1}$ where $C'$ and $J'$ is reliably generated normal form (using sympy built in function)

In [ ]:
for i in range(100):
  test = generate_nice_matrix(5,10)
  result = jordan_form(test).jordan_form()[1]
  P, J = test.jordan_form()

  print(f"test {i+1}")
  assert result.equals(J)

test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9
test 10
test 11
test 12
test 13
test 14
test 15
test 16
test 17
test 18
test 19
test 20
test 21
test 22
test 23
test 24
test 25
test 26
test 27
test 28
test 29
test 30
test 31
test 32
test 33
test 34
test 35
test 36
test 37
test 38
test 39
test 40
test 41
test 42
test 43
test 44
test 45
test 46
test 47
test 48
test 49
test 50
test 51
test 52
test 53
test 54
test 55
test 56
test 57
test 58
test 59
test 60
test 61
test 62
test 63
test 64
test 65
test 66
test 67
test 68
test 69
test 70
test 71
test 72
test 73
test 74
test 75
test 76
test 77
test 78
test 79
test 80
test 81
test 82
test 83
test 84
test 85
test 86
test 87
test 88
test 89
test 90
test 91
test 92
test 93
test 94
test 95
test 96
test 97
test 98
test 99
test 100


Computing the change of bases:

We aim to find the generalized vectors in the space $ H_n(\lambda) $. Find the basis of vectors such that any vector is in $ H_n(\lambda) $ but not in $ H_{n-1}(\lambda) $. Denote $ V = H_n(\lambda) $ and $ U = H_{n-1}(\lambda) \subset V $. Let $ B_V = (v_1, \dots, v_n) $ be ONB of $ V $ and $ B_U = (u_1, \dots, u_m) $ be an ONB of $ U $. There always exists the decomposition $ U \oplus U^\perp = V $.

Proceed with finding the basis of $ U^\perp $ as follows:

1) Define $ \pi_U(v) = \sum_{i=1}^m \langle v, u_i \rangle u_i, \pi_U \in \text{Hom}(V, U) $ — orthogonal projection onto the space $ U $.

 For each $ v \in V $, there exist $ u \in U $ and $ u' \in U^\perp $ such that $ u + u' = v $. Then $ \pi_U(v) = \pi_U(u + u') = u' \in U^\perp $. Then $ w_i = v_i - \pi_U(v_i) \in U^\perp $.

2\) Show that $ w_1, \dots, w_n $ is the spanning set of $ U^\perp $.

Let $ x \in U^\perp \subset V $. There exists a linear combination of the vectors in $ B_V $ such that $ \sum_{i=1}^n a_i v_i = x $.

Take $ \pi_U $ on both sides:
$$
\pi_U \left( \sum_{i=1}^n a_i v_i \right) = \sum_{i=1}^n a_i \pi_U (v_i) = \pi_U(x) = 0 \text{ since } x \in U^\perp
$$
$$
\sum_{i=1}^n a_i w_i = \sum_{i=1}^n a_i (v_i - \pi_U(v_i)) = \sum_{i=1}^n a_i v_i - \sum_{i=1}^n a_i \pi_U (v_i) = x - 0 = x
$$

For $ \forall x \in U^\perp $, there exists a linear combination $ x = \sum_{i=1}^n a_i w_i \Rightarrow \text{span} \{w_1, \dots, w_n\} = U^\perp $.

Start with any $ w_k \neq 0 $. Then since $ w_1, \dots, w_n $ is a generating set, we can complement the vector $ w_k $ to form a basis (basis extension theorem).

3) Basis extraction procedure: We must find $\dim V - \dim U$ linearly independent vectors in $ w_1, \dots, w_n $.

Apply the Gauss method to find a row echelon form of the matrix $ (w_1 | \dots | w_n) $. Then the number of steps is equal to the number of linearly independent vectors (the Gauss method can be represented as multiplication by an invertible matrix, which does not change the rank of the original matrix). Moreover, indices at which the steps occur point exactly to those vectors which are linearly independent.


In [ ]:
def lin_independent(B):
    echelon_form = sp.Matrix.hstack(*B).rref()
    return [B[i] for i in echelon_form[1]]


def disjoint(V, U):
    disjoint_ = []
    for v in V:
        projection = sp.zeros(v.shape[0], v.shape[1])
        for u in U:
            projection = projection + v.dot(u) * u
        disjoint_.append(v - projection)
    return lin_independent(disjoint_)


def length(v):
    return sp.sqrt(v.dot(v))


def orthonormalize(B):
    if len(B) == 0:
        return []

    orthonormal = [B[0] / length(B[0])]
    for b in B[1:]:
        projection = sp.zeros(b.shape[0], b.shape[1])
        for o in orthonormal:
            projection = projection + b.dot(o) * o
        diff = b - projection
        orthonormal.append(diff / length(diff))
    return orthonormal


def ranks(matrix: sp.Matrix):
    ranks_ = [
        matrix.shape[0]
    ]  # dim  E(lambda) = n-dim Im(A-lambda)=n-rk (A-lambda)
    bases_ = [[]]  # dim Ker(Id) = 0
    current_matrix = sp.eye(matrix.shape[0])
    while True:
        current_rank = current_matrix.rank()
        next_matrix = current_matrix * matrix
        if current_rank == next_matrix.rank():
            break
        ranks_.append(next_matrix.rank())
        bases_.append(orthonormalize(next_matrix.nullspace()))
        current_matrix = next_matrix
    return (ranks_[::-1], bases_[::-1])


def chains(matrix, ranks, bases):
    chain_count = 0
    chains_sizes_ = []
    chains_ = [[]]

    for i in range(len(ranks) - 1):
        d = ranks[i + 1] - ranks[i]
        chains_cont = [matrix * vector for vector in chains_[len(chains_) - 1]]
        disjoint_basis = disjoint(bases[i], bases[i + 1])

        if d > chain_count:
            new_chains = disjoint(
                orthonormalize(disjoint_basis), orthonormalize(chains_cont)
            )
            chains_cont.extend(new_chains)
            chains_sizes_.append((d - chain_count, len(ranks) - 1 - i))
            chain_count = d
        chains_.append(chains_cont)

    return (chains_sizes_, chains_)


def collect_chain(chains, i):
    chain = []
    j = len(chains) - 1
    while j >= 0 and len(chains[j]) > i:
        chain.append(chains[j][i])
        j -= 1
    return chain


def collect_chains(chains):
    collected = []
    for i in range(len(chains[len(chains) - 1])):  # starting from eigenvectors
        chain = collect_chain(chains, i)
        collected.extend(chain)
    return collected


def jordan_form(matrix: sp.Matrix, raw=False):
    size = matrix.shape[0]
    blocks = []
    raw_blocks = []
    evas = matrix.eigenvals().keys()
    collected_chains = []
    for eva in evas:
        nilpotent = matrix - eva * sp.eye(
            size
        )  # nilpotent but this doesnt tell much
        ranks_, bases_ = ranks(nilpotent)
        chains_sizes_, chains_ = chains(nilpotent, ranks_, bases_)
        collected_chains.extend(collect_chains(chains_))
        for chain in chains_sizes_:
            chain_size = chain[1]
            block = sp.jordan_cell(eva, chain_size)
            raw_blocks.extend([(eva, chain_size)] * chain[0])
            blocks.extend([block] * chain[0])
    if not raw:
        return (
            sp.Matrix(block_diag(*blocks)),
            sp.Matrix.hstack(*collected_chains),
        )
    else:
        return (raw_blocks, sp.Matrix.hstack(*collected_chains))


def euclidian_vector(size, index):
    vector = np.zeros(size)
    vector[index] = 1
    return vector


def raise_block(eva, size, power):
    columns = [eva * euclidian_vector(size, 0)]
    columns.extend(
        [
            eva * euclidian_vector(size, i) + euclidian_vector(size, i - 1)
            for i in range(1, size)
        ]
    )
    for p in range(power - 1):
        prev = columns[0].copy()
        columns[0] = eva * columns[0]
        for i in range(1, size):
            new_prev = columns[i].copy()
            columns[i] = eva * columns[i] + prev
            prev = new_prev
    return np.vstack(columns).T


def raise_matrix(matrix: sp.Matrix, power):
    J, P = jordan_form(matrix, True)
    P = sympy_to_numpy(P)
    blocks = [raise_block(block[0], block[1], power) for block in J]

    return P @ block_diag(*blocks) @ np.linalg.inv(P)


A = sp.Matrix(
    np.array(
        [
            [3, 1, -1, 1, -1],
            [-1, 5, 1, -1, 1],
            [-1, 1, 3, 1, -1],
            [0, 0, 0, 4, 2],
            [-1, 1, 1, -1, 7],
        ]
    )
)

J, P = jordan_form(A)

print(J)
print(P * J * P.inv())
print(raise_matrix(A, 3))

Matrix([[2, 0, 0, 0, 0], [0, 6, 1, 0, 0], [0, 0, 6, 0, 0], [0, 0, 0, 4, 0], [0, 0, 0, 0, 4]])
Matrix([[3, 1, -1, 1, -1], [-1, 5, 1, -1, 1], [-1, 1, 3, 1, -1], [0, 0, 0, 4, 2], [-1, 1, 1, -1, 7]])
[[36.0000000000000 28.0000000000000 -28.0000000000000 28.0000000000000
  -28.0000000000000]
 [-76.0000000000000 140.000000000000 76.0000000000000 -76.0000000000000
  76.0000000000000]
 [-28.0000000000000 28.0000000000000 36.0000000000000 28.0000000000000
  -28.0000000000000]
 [-32.0000000000000 32.0000000000000 32.0000000000000 32.0000000000000
  184.000000000000]
 [-108.000000000000 108.000000000000 108.000000000000 -108.000000000000
  324.000000000000]]


Testing basis change:

In [ ]:
for i in range(100):
  test = generate_nice_matrix(5,10)
  J,P = jordan_form(test)
  J_np, P_np = sympy_to_numpy(J), sympy_to_numpy(P)
  restored = P_np @ J_np @ np.linalg.inv(P_np)

  print(f"test {i+1}")
  assert np.allclose(restored, sympy_to_numpy(test))

test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9
test 10
test 11
test 12
test 13
test 14
test 15
test 16
test 17
test 18
test 19
test 20
test 21
test 22
test 23
test 24
test 25
test 26
test 27
test 28
test 29
test 30
test 31
test 32
test 33
test 34
test 35
test 36
test 37
test 38
test 39
test 40
test 41
test 42
test 43
test 44
test 45
test 46
test 47
test 48
test 49
test 50
test 51
test 52
test 53
test 54
test 55
test 56
test 57
test 58
test 59
test 60
test 61
test 62
test 63
test 64
test 65
test 66
test 67
test 68
test 69
test 70
test 71
test 72
test 73
test 74
test 75
test 76
test 77
test 78
test 79
test 80
test 81
test 82
test 83
test 84
test 85
test 86
test 87
test 88
test 89
test 90
test 91
test 92
test 93
test 94
test 95
test 96
test 97
test 98
test 99
test 100


Testing raising matrix to the power:

In [ ]:
for i in range(100):
  test = generate_nice_matrix(5,10)
  raised = raise_matrix(test, 5)
  comparison = sympy_to_numpy(test**5)

  print(f"test {i+1}")
  assert np.allclose(raised, comparison)

test 1
test 2
test 3
test 4
test 5
test 6
test 7
test 8
test 9
test 10
test 11
test 12
test 13
test 14
test 15
test 16
test 17
test 18
test 19
test 20
test 21
test 22
test 23
test 24
test 25
test 26
test 27
test 28
test 29
test 30
test 31
test 32
test 33
test 34
test 35
test 36
test 37
test 38
test 39
test 40
test 41
test 42
test 43
test 44
test 45
test 46
test 47
test 48
test 49
test 50
test 51
test 52
test 53
test 54
test 55
test 56
test 57
test 58
test 59
test 60
test 61
test 62
test 63
test 64
test 65
test 66
test 67
test 68
test 69
test 70
test 71
test 72
test 73
test 74
test 75
test 76
test 77
test 78
test 79
test 80
test 81
test 82
test 83
test 84
test 85
test 86
test 87
test 88
test 89
test 90
test 91
test 92
test 93
test 94
test 95
test 96
test 97
test 98
test 99
test 100
